#  Introdução ao Python
## Professor: Luiz Ferreira

## Módulo 1

### Introdução ao Python

_____________

## Backend

Após o desenvolvimento da parte frontal de nossa aplicação, precisamos dar vida aos recursos moldados. O trabalhar a parte invisível ao usuário é chamada de backend. Nessa parte de nossa aula, trabalharemos nas ações de cada elemento afim de possibilitar o uso da aplicação juntamente com o banco de dados relacionado.

### Vale lembrar:
Para o desenvolvimento da aplicação, necessitamos criar uma tabela no banco de dados que contenha os mesmos elementos descritos durante o desenvolvimento do Frontend! A lembrar:

- id
- nome
- sobrenome
- email
- cpf

Com isso podemos salvar todos as informações preenchidas no frontend.

### Vale lembrar: 
Existe uma estrutura mínima para que possamos trabalhar o nosso banco de dados em SQLite! A lembrar:

In [ ]:
import sqlite3
#Conectando ao banco de daods...
conn = sqlite3.connect(database)
cur = conn.cursor()
 
#... Ações usando o banco...
 
#Fazendo commit e desconecta do banco...
conn.commit()
conn.close()

Como esse código poderá repetir diversas vezes em nossa aplicação, podemos utilizar o conceito de classe aprendido em aulas anteriores e fazer de nosso código uma estrutura amigável e reutilizável. Para isso:

In [ ]:
import sqlite3 as sql

class TransactionObject():        #Nome de nossa classe, que não necessita de parametros, inicialmente
    database    = "clientes.db"   #Nome e local do banco de dados
    conn        = None            #Variável que receberá nossa conexão
    cur         = None            #Variável que receberá o cursor (visto na aula passada)
    connected   = False           #Simples checagem de estado, conectado ou não
 
    def connect(self):            #Função que realiza a conexão no banco de dados
        TransactionObject.conn      = sql.connect(TransactionObject.database)
        TransactionObject.cur       = TransactionObject.conn.cursor()
        TransactionObject.connected = True
 
    def disconnect(self):         #Função que fecha nossa conexão
        TransactionObject.conn.close()
        TransactionObject.connected = False
 
    def execute(self, sql, parms = None):  #Função que executa alguma ação
        if TransactionObject.connected:
            if parms == None:
                TransactionObject.cur.execute(sql)
            else:
                TransactionObject.cur.execute(sql, parms)
            return True
        else:
            return False
 
    def fetchall(self):           #Função que constrói um objeto no qual podemos recuperar nossas informações
        return TransactionObject.cur.fetchall()
 
    def persist(self):            #Função que realiza o commit, ou seja, grava no nosso banco de dados
        if TransactionObject.connected:
            TransactionObject.conn.commit()
            return True
        else:
            return False

### Relembrando

- connect(): realiza conexão com o banco de dados
- disconnect(): fecha a conexão com o banco de dados
- execute(self, sql, parms): executa um comando no banco de dados. recebe três parâmetros:
- self: referencia para o próprio objeto. não precisa ser informado;
- sql: comando SQL a ser executado;
- parms: vetor com os parâmetros do comando SQL. Pode ser omitido.
- fetchall(): recupera os valores recebidos de um comando select.
- persist(): realiza o commit das operações realizadas.

## Continuando ...

Após construir nossa classe de conexão (isso vai nos ajudar demais).

Passamos para nossa classe `backend` propriamente dita!

Quando iniciarmos nossa aplicação, necessitamos conectar no banco e garantir que nossa tabela existe.

In [ ]:
def initDB():
    trans = TransactionObject()
    trans.connect()
    trans.execute("CREATE TABLE IF NOT EXISTS clientes (id INTEGER PRIMARY KEY , nome TEXT, sobrenome TEXT, email TEXT, cpf TEXT)")
    trans.persist()
    trans.disconnect()
 
initDB()

No trecho de código acima, criamos a função initDB() e, logo após, criamos uma chamada dessa função. Com isso, é possível fazer com que a cada vez que a nossa aplicação seja iniciada, o banco então é checado e, caso não exista a tabela, ele a criará.

In [ ]:
def view():
    trans = TransactionObject()
    trans.connect()
 
    trans.execute("SELECT * FROM clientes")
 
    rows = trans.fetchall()
    trans.disconnect()
    return rows

A função view() será responsável por recuperar todos os clientes que já se encontram no banco de dados, para que possamos exibir posteriormente.

In [ ]:
def insert(nome, sobrenome, email, cpf):
    trans = TransactionObject()
    trans.connect()
    trans.execute("INSERT INTO clientes VALUES(NULL, ?,?,?,?)", (nome, sobrenome, email, cpf))
    trans.persist()
    trans.disconnect()

A função insert é a responsável por gravar um novo valor dentro de nosso banco de dados, observe que essa função necessita de 4 parâmetros, que são as informações que foram solicitadas para o usuário.

In [ ]:
def search(nome="", sobrenome="", email="", cpf=""):
    trans = TransactionObject()
    trans.connect()
    trans.execute("SELECT * FROM clientes WHERE nome=? or sobrenome=? or email=? or cpf=?", (nome,sobrenome,email, cpf))
    rows = trans.fetchall()
    trans.disconnect()
    return rows

A função search é a responsável por buscar os clientes já cadastrados.

In [ ]:
def update(id, nome, sobrenome, email, cpf):
    trans = TransactionObject()
    trans.connect()
    trans.execute("UPDATE clientes SET nome =?, sobrenome=?, email=?, cpf=? WHERE id = ?",(nome, sobrenome,email, cpf, id))
    trans.persist()
    trans.disconnect()

A função update atualizará registros dentro do banco de dados.

In [ ]:
def delete(id):
    trans = TransactionObject()
    trans.connect()
    trans.execute("DELETE FROM clientes WHERE id = ?", (id,))
    trans.persist()
    trans.disconnect()

E, por último, temos a função delete que receberá o `id` como parâmetro e realizará a exclusão desse registro.

## E agora?

Temos o `Frontend` que faz a comunicação com o usuário, temos o `Backend` que é o código que realizará as ações de cada botão. Temos também nossa classe de conexão, que permite criar, atualizar, deletar e consultar nossa base de dados.

Com isso podemos fazer a conexão entre `Frontend` e `Backend`!